In [1]:
# - *- coding: utf-8*-
'''
    Used Resources:
        => https://www.tensorflow.org/api_docs/python/tf
'''

import numpy as np
import tensorflow as tf
from keras import backend as K

#TODO missing documentation and maybe try catch of keras backend!
#TODO maybe test needed
#TODO maybe create a class 

'''
    This class library allow to use tensorflow api, 
    to get the direct node neigbour embedding of graph nodes 
    depending on the nodes feature vectors.
'''

#These are the edges between the nodes.
current = np.array( [
                    [0,0.3,0.3,1,0.3],
                    [0.1,0.3,1,0,0.2],
                    [0,0.7,1,1,0.4],
                    [0.8,0.2,1,1,0.2]
                    ], dtype='float32')

current_ten = tf.keras.backend.variable (value=current, dtype='float32')

################################

neigbours = np.array([
                     [0,1,1,1],
                     [0,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0]
                     ], dtype='float32')

neighbours_ten = tf.keras.backend.variable (value=neigbours, dtype='float32')

print('##### Load Variables #####')

##### Load Variables #####


Using TensorFlow backend.


In [ ]:
def GetMaxColIndexFromTensor(tensor_x):
    count = tensor_x.get_shape()[0]
    return tf.convert_to_tensor(count)

def GetColCount2DTensorKeras(tensor):
    return K.variable(value=tensor.shape[0], dtype='int32')

def GetMaxRowsIndexFromTensor(tensor_x):
    cols = tensor_x.get_shape()[1]
    return tf.convert_to_tensor(cols)

def GetNoneZerosOfVecTensor(vec_tens):
    return tf.convert_to_tensor(tf.count_nonzero(vec_tens, dtype=tf.float32))   

def GetSummedNeighboursByEdges(neigh_emb, feature_tensor):
    return tf.reduce_sum(tf.multiply(tf.expand_dims(neigh_emb,-1), feature_tensor), axis=0)

# This function return the tensor rank as tensor constant
def GetMaxRankFromTensor(tensor_x):
    return tf.constant(tf.rank(tensor_x).eval())

# This function return the tensors shape as list/narray
def GetShapeListFromTensor(tensor_x):
    return tensor_x.get_shape().as_list()

def KerasEval(tensor_x):
    return tf.keras.backend.eval(tensor_x)

def KerasShape(tensor_x):
    return tf.keras.backend.shape(tensor_x)

def GetDefaultFloat32TensArray():
    return tf.TensorArray(dtype=tf.float32, infer_shape=False, size=1, dynamic_size=True)

print('##### Load Basic Functions #####')

In [2]:
def GetNeighbourEmbeddingWithMean(edge_emb, feature_vec_tensor):
    sum_tensor = GetSummedNeighboursByEdges(edge_emb, feature_vec_tensor)
    divider = GetNoneZerosOfVecTensor(edge_emb)
    return GetMeanOrNull(edge_emb, sum_tensor, divider)

def GetMeanOrNull(edge_emb, sum_tensor, divider):
    return tf.cond(tf.count_nonzero(edge_emb) > 0, 
                   lambda: tf.divide(sum_tensor, divider),
                   lambda: sum_tensor)



def GetNextNodeFeatureTensors(edge_emb, node_emb, result_emb, iterator):
    
    #Condition => if current iteration is lower as max iterations
    def cond(edge_emb, node_emb, result_emb, i):
         return K.less(i, iterator)

    #Loop Body Processing Unit => get all connection dependencies
    def body(edge_emb, node_emb, result_emb, i):
        
        #Get current viewed Node Neighbour embedding mean as result!
        result = GetNeighbourEmbeddingWithMean(edge_emb[i, :], node_emb)
        
        #Store result in Tensorarray result_emb
        result_emb = result_emb.write(i, [result])
        
        #Add +1 to Iterator and return process result!
        return [edge_emb, node_emb, result_emb, tf.add(i, 1)]

    #return the loop setup
    return tf.while_loop(cond, body, [edge_emb, node_emb, result_emb, 0])

def GetTensorResults(edges, nodes, result_tens_arry, iterator):
    res = GetNextNodeFeatureTensors(edges, nodes, result_tens_arry, iterator)
    return res[0], res[1], res[2].concat(), res[3]

def GetFeaturesTensorOnly(edge_and_nodes):
    iterations = GetColCount2DTensorKeras(edge_and_nodes[0])
    carrier = GetDefaultFloat32TensArray()    
    result = GetTensorResults(edge_and_nodes[0], edge_and_nodes[1], carrier, iterations)[2]
    print(result)
    return result 

print('##### Load Functions #####')

##### Load Functions #####


In [3]:
print('##### Execute Tests #####')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    GetFeaturesTensorOnly([current_ten, neighbours_ten])

##### Execute Tests #####


ValueError: Dimensions must be equal, but are 5 and 4 for 'while/Mul' (op: 'Mul') with input shapes: [5,1], [4,4].